In [30]:
import cv2 as cv
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
openFpath = input(r"Open rbnir image file: ")
with rio.open(openFpath) as dst:
    satImg = dst.read()
    satMeta = dst.meta

In [4]:
newImg = np.dstack((satImg[0], satImg[1], satImg[2], satImg[3]))
newImg.shape

(1073, 1080, 4)

In [6]:
satz = newImg.reshape((-1, 4))
satz.shape

(1158840, 4)

In [11]:
satz = np.float32(satz)

In [8]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
print(cv.TERM_CRITERIA_EPS)
print(cv.TERM_CRITERIA_MAX_ITER)

2
1


In [12]:
k = 2
see = []
labels = []
knos = []
maxCluster = 30
while k <= maxCluster:
    ret,label,_=cv.kmeans(satz,k,None,criteria,10,cv.KMEANS_RANDOM_CENTERS)
    see.append(ret)
    knos.append(k)
    labels.append(label)
    k+=1

In [19]:
label.shape

(1158840, 1)

In [21]:
labelImg = label.reshape(newImg.shape[:2])

In [22]:
labelImg.shape

(1073, 1080)

In [23]:
plt.imshow(labelImg)

In [24]:
plt.show()

In [25]:
np.unique(label)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int32)

In [26]:
labelImg+= 1

dtype('int32')

In [28]:
labelImg = np.uint16(labelImg)

In [31]:
destFolder = input(r"Destination folder of label file: ")
destFpath = Path(destFolder, "cluster_kmeans_30.tif")

In [35]:
satMeta.update(count = 1)

In [43]:
with rio.open(destFpath, 'w', **satMeta) as dst:
    dst.write(labelImg, 1)